In [1]:
import duckdb as db
import pandas as pd

In [2]:
def sql(query, database = "../data/database.duckdb"):
    con = db.connect(database)
    res = con.execute(query).df()
    con.close()
    return res

In [18]:
urls = {
    'name_basics' : 'https://datasets.imdbws.com/name.basics.tsv.gz',
    'title_basics' : 'https://datasets.imdbws.com/title.basics.tsv.gz',
    'title_crew' : 'https://datasets.imdbws.com/title.crew.tsv.gz',
    'title_ratings' : 'https://datasets.imdbws.com/title.ratings.tsv.gz',
    'title_akas' : 'https://datasets.imdbws.com/title.akas.tsv.gz',
    'tmdb' : '../data/tmdb_full.csv'
}

for name, url in urls.items():
    if name == 'title_basics':
        sql(
            f"""
                create or replace table {name} as
                select * from read_csv_auto('{url}')
                where titletype = 'movie'
            """
        )
    else:
        sql(
            f"""
                create or replace table {name} as
                select * from read_csv_auto('{url}')
            """
        )


In [20]:
sql(
            f"""
                create or replace table title_principals as
                select * from read_csv_auto('https://datasets.imdbws.com/title.principals.tsv.gz')
            """
        )

,Count
0,92760677


In [5]:
sql(
    """
    show tables

"""
)

,name
0,dataframe_view
1,name_basics
2,title_akas
3,title_basics
4,title_crew
5,title_principals
6,title_ratings
7,tmdb


In [4]:
df = pd.DataFrame(
    sql('''
        SELECT * FROM dataframe_view
        '''
    )
)

In [6]:
df.to_parquet('../data/app_df.gzip', compression='gzip')

In [7]:
df_test = pd.read_parquet('../data/app_df.gzip')

In [4]:
pd.read_parquet("../data/df_similar.gzip")
df = pd.read_parquet("../data/df_similar.gzip")
df.head()

,tconst,Title,Genres,Year,Runtime,Actors,Director,Writers,Resume,Rating,Votes,ProductionCompanies,ProductionCountries,Poster,similar_films
0,tt0499549,Avatar,"Action,Adventure,Fantasy",2009,162,"Wes Studi, Joel David Moore, CCH Pounder, Mich...",James Cameron,James Cameron,"In the 22nd century, a paraplegic Marine is di...",7.9,1430785,"Dune Entertainment, Lightstorm Entertainment, ...","US, GB",/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg,"[22, 63, 46, 145, 50]"
1,tt4154796,Avengers: Endgame,"Action,Adventure,Sci-Fi",2019,181,"Don Cheadle, Chris Evans, Robert Downey Jr., C...","Joe Russo, Anthony Russo","Stan Lee, Bill Mantlo, Don Heck, Joe Simon, La...",After the devastating events of Avengers: Infi...,8.4,1357117,Marvel Studios,Unknown,/or06FN3Dka5tukK1e9sl16pB3iy.jpg,"[5, 33, 22, 12, 21]"
2,tt0120338,Titanic,"Drama,Romance",1997,194,"Kate Winslet, Frances Fisher, Gloria Stuart, K...",James Cameron,James Cameron,101-year-old Rose DeWitt Bukater tells the sto...,7.9,1339359,"Paramount, 20th Century Fox, Lightstorm Entert...",US,/9xjZS2rlVxm8SFx8kPC3aIGCOYQ.jpg,"[0, 22, 348, 524, 1353]"
3,tt1630029,Avatar: The Way of Water,"Action,Adventure,Fantasy",2022,192,"CCH Pounder, Brendan Cowell, Sigourney Weaver,...",James Cameron,"Amanda Silver, Shane Salerno, Josh Friedman, J...",Set more than a decade after the events of the...,7.5,534023,"20th Century Studios, Lightstorm Entertainment",Unknown,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,"[96, 123, 275, 79041, 2]"
4,tt2488496,Star Wars: Episode VII - The Force Awakens,"Action,Adventure,Sci-Fi",2015,138,"John Boyega, Carrie Fisher, Andy Serkis, Daisy...",J.J. Abrams,"J.J. Abrams, Lawrence Kasdan, George Lucas, Mi...",Thirty years after defeating the Galactic Empi...,7.8,999627,"Lucasfilm Ltd., Bad Robot",Unknown,/wqnLdwVXoBjKibFRR5U3y0aDUhs.jpg,"[145, 63, 94, 50, 22]"


In [ ]:
df['Year'] = pd.to_datetime(df['Year'])
df.info()
df.to_csv("../data/df.csv") 

<class 'pandas.core.frame.DataFrame'>
Index: 99312 entries, 0 to 101543
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   tconst               99312 non-null  object        
 1   Title                99312 non-null  object        
 2   Genres               99312 non-null  object        
 3   Year                 99312 non-null  datetime64[ns]
 4   Runtime              99312 non-null  int64         
 5   Actors               99312 non-null  object        
 6   Director             99312 non-null  object        
 7   Writers              99312 non-null  object        
 8   Resume               99312 non-null  object        
 9   Rating               99312 non-null  float64       
 10  Votes                99312 non-null  int64         
 11  ProductionCompanies  99312 non-null  object        
 12  ProductionCountries  99312 non-null  object        
 13  Poster               99312 non-null

In [ ]:
pd.read_parquet("../data/df_similar.gzip")
df = pd.read_parquet("../data/df_similar.gzip")
df.replace({r'[\r\n]+': ' '}, regex=True, inplace=True)
df.to_csv("../data/df.csv", index=False, sep=';', encoding='utf-8-sig')